Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
entropy_func = tf.nn.softmax_cross_entropy_with_logits

# Parameters
batch_size = 128
l2reg_beta = 0.001

graph = tf.Graph()
with graph.as_default():

    # Input data. Use placeholder for minibatch training data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(entropy_func(logits, tf_train_labels)) + l2reg_beta*tf.nn.l2_loss(weights)
  
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 4001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print("Initialized")
    for step in range(num_steps):
        # Generate a minibatch.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        # Run the training
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % (num_steps/5) == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 20.818031
Minibatch accuracy: 7.8%
Validation accuracy: 10.5%
Minibatch loss at step 800: 1.751947
Minibatch accuracy: 83.6%
Validation accuracy: 77.6%
Minibatch loss at step 1600: 1.279204
Minibatch accuracy: 73.4%
Validation accuracy: 80.6%
Minibatch loss at step 2400: 0.801474
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 3200: 0.607909
Minibatch accuracy: 84.4%
Validation accuracy: 81.5%
Minibatch loss at step 4000: 0.656199
Minibatch accuracy: 82.8%
Validation accuracy: 82.4%
Test accuracy: 89.0%


In [7]:
# Parameters
hidden_nodes = 1024

# Define the TF graph
graph = tf.Graph()
with graph.as_default():

    # Mini-batch training set, validation set, test set
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Hidden layer weights
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    # Hidden layer computation
    hidden_logits = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
    hidden_relus = tf.nn.relu(hidden_logits)
    # Output layer weights
    output_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))
    # Output layer computation
    output_logits = tf.matmul(hidden_relus, output_weights) + output_biases
    loss = (tf.reduce_mean(entropy_func(output_logits, tf_train_labels)) +
            l2reg_beta*tf.nn.l2_loss(hidden_weights) +
            l2reg_beta*tf.nn.l2_loss(output_weights))

    # Setup the optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Training dataset predictions
    train_prediction = tf.nn.softmax(output_logits)
    # Validation dataset predictions
    valid_hiddens = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hiddens, output_weights) + output_biases)
    # Test dataset predictions
    test_hiddens = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_hiddens, output_weights) + output_biases)

In [8]:
num_steps = 3000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):

        # Generate the minibatch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        # Run the computations
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % (num_steps/5) == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 633.350769
Training accuracy: 9.4%
Validation accuracy: 27.0%
Loss at step 600: 176.213898
Training accuracy: 82.8%
Validation accuracy: 80.5%
Loss at step 1200: 92.831215
Training accuracy: 87.5%
Validation accuracy: 82.3%
Loss at step 1800: 50.610935
Training accuracy: 79.7%
Validation accuracy: 83.7%
Loss at step 2400: 27.727049
Training accuracy: 88.3%
Validation accuracy: 85.4%
Test accuracy: 93.0%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):

        # Generate the minibatch
        batchmod = min(batch_size*2, train_labels.shape[0] - batch_size)
        offset = (step * batch_size) % batchmod        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        # Run the computations
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % (num_steps/5) == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 659.829224
Training accuracy: 8.6%
Validation accuracy: 33.1%
Loss at step 600: 172.275650
Training accuracy: 100.0%
Validation accuracy: 73.6%
Loss at step 1200: 94.532585
Training accuracy: 100.0%
Validation accuracy: 73.6%
Loss at step 1800: 51.872852
Training accuracy: 100.0%
Validation accuracy: 73.7%
Loss at step 2400: 28.464216
Training accuracy: 100.0%
Validation accuracy: 73.7%
Loss at step 3000: 15.619463
Training accuracy: 100.0%
Validation accuracy: 73.5%
Test accuracy: 80.9%


When we only use a few batches, the model becomes highly over trained on those samples and doesn't generalize well. The L2 regularization probably helps here a fair amount.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [13]:
# Define the neural network graph again

# Parameters
hidden_nodes = 1024
l2reg_beta = 0

# Define the TF graph
graph = tf.Graph()
with graph.as_default():

    # Mini-batch training set, validation set, test set
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Hidden layer weights
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))

    # Hidden layer computation with dropout
    keep_prob = tf.placeholder(tf.float32)
    hidden_logits = tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases
    hidden_relus = tf.nn.relu(hidden_logits)
    hidden_relus_drop = tf.nn.dropout(hidden_relus, keep_prob)
    # Output layer weights
    output_weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    output_biases = tf.Variable(tf.zeros([num_labels]))
    # Output layer computation
    output_logits = tf.matmul(hidden_relus_drop, output_weights) + output_biases
    loss = (tf.reduce_mean(entropy_func(output_logits, tf_train_labels)) +
            l2reg_beta*(tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(output_weights)))

    # Setup the optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    # Training dataset predictions
    #train_prediction = tf.nn.softmax(output_logits)
    train_hiddens = tf.nn.relu(tf.matmul(tf_train_dataset, hidden_weights) + hidden_biases)
    train_prediction = tf.nn.softmax(tf.matmul(train_hiddens, output_weights) + output_biases)
    # Validation dataset predictions
    valid_hiddens = tf.nn.relu(tf.matmul(tf_valid_dataset, hidden_weights) + hidden_biases)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hiddens, output_weights) + output_biases)
    # Test dataset predictions
    test_hiddens = tf.nn.relu(tf.matmul(tf_test_dataset, hidden_weights) + hidden_biases)
    test_prediction = tf.nn.softmax(tf.matmul(test_hiddens, output_weights) + output_biases)

In [14]:
# TODO: not sure how, but I need to steer the dropout keep_prob from here so that it is
# 0.5 while training and 1.0 while evaluating. Hmmm.


num_steps = 3000

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):

        # Generate the minibatch
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                     keep_prob : 0.5}
        
        # Run the computations
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % (num_steps/5) == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))

    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 481.858582
Training accuracy: 9.4%
Validation accuracy: 33.5%
Loss at step 600: 28.382696
Training accuracy: 79.7%
Validation accuracy: 79.9%
Loss at step 1200: 7.394904
Training accuracy: 81.2%
Validation accuracy: 79.2%
Loss at step 1800: 5.798740
Training accuracy: 75.0%
Validation accuracy: 79.4%
Loss at step 2400: 5.945658
Training accuracy: 80.5%
Validation accuracy: 78.8%
Test accuracy: 87.8%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
